In [1]:
from copy import deepcopy

"""
Symbol:           H     B     C     N     O     F    Mg     P     S    Cl    Br
Valence number:   1     3     4     3     2     1     2     3     2     1     1
Atomic weight:  1.0  10.8  12.0  14.0  16.0  19.0  24.3  31.0  32.1  35.5  80.0  (in g/mol)
"""

elements = {
    "H": (1, 1.0),
    "B": (3, 10.8),
    "C": (4, 12.0),
    "N": (3, 14.0),
    "O": (2, 16.0),
    "F": (1, 19.0),
    "Mg": (2, 24.3),
    "P": (3, 31.0),
    "S": (2, 32.1),
    "Cl": (1, 35.5),
    "Br": (1, 80.0),
}


class InvalidBond(Exception):
    pass


class UnlockedMolecule(Exception):
    pass


class LockedMolecule(Exception):
    pass


class EmptyMolecule(Exception):
    pass


class Atom(object):

    def __init__(self, elt, id_):
        self.element = elt
        self.id = id_
        self.valence = elements[elt][0]
        self.valence_free = elements[elt][0]
        self.weight = elements[elt][1]
        self.bonds: list[Atom] = []

    def __hash__(self):
        return self.id

    def __eq__(self, other):
        return self.id == other.id

    def __str__(self) -> str:
        """Return a string formatted like the following:
        "Atom(element.id: element1id,element2id,element3id...)".

        element: symbol of the current Atom instance
        id: id of the current element (beginning at 1 for each Molecule instance)
        element1id: element1, bonded to the current Atom and its id number.
        If the bonded atom is a hydrogen, do not display its id number, to increase readability.
        """
        key = lambda atom: (
            ("?", atom.id)
            if atom.element == "C"
            else (("z", atom.id) if atom.element == "H" else (atom.element, atom.id))
        )
        self.bonds.sort(key=key)

        bonded = [
            f"{atom.element}{atom.id}" if atom.element != "H" else f"{atom.element}"
            for atom in self.bonds
        ]

        return (
            f"Atom({self.element}.{self.id}{': ' if bonded else ''}{','.join(bonded)})"
        )

    def bond(self, other):
        if self.valence_free == 0 or other.valence_free == 0 or self == other:
            raise InvalidBond
        self.bonds.append(other)
        self.valence_free -= 1

        other.bonds.append(self)
        other.valence_free -= 1

    def mutate(self, elt):
        if elements[elt][0] - len(self.bonds) < 0:
            raise InvalidBond
        self.element = elt
        self.valence = elements[elt][0]
        self.valence_free = elements[elt][0] - len(self.bonds)
        self.weight = elements[elt][1]

    def unbond(self):
        for atom in self.bonds:
            atom.bonds.remove(self)
            atom.valence_free += 1
        self.bonds = []
        self.valence_free = self.valence


class Molecule(object):
    def __init__(self, name="") -> None:
        self.name = name
        self.__formula = ""
        self.__molecular_weight = 0
        self.atoms = []
        self.__branches = []
        self.locked = False

    def brancher(self, *args):
        """In a Molecule instance, a "branch" represents a chain of atoms bounded together. When a branch is created, all of its atoms are carbons. Each "branch" of the Molecule is identifiable by a number that matches its creation order: first created branch as number 1, second as number 2, ...

        The brancher method...:

        Can take any number of arguments (positive integers).
        Adds new "branches" to the current molecule.
        Each argument gives the number of carbons of the new branch."""
        if self.locked:
            raise LockedMolecule

        for i in args:
            carbons = [
                Atom("C", id)
                for id in range(len(self.atoms) + 1, len(self.atoms) + 1 + i)
            ]
            for j in range(1, i):
                carbons[j - 1].bond(carbons[j])
            self.atoms.extend(carbons)
            self.__branches.append(carbons)

        return self

    def add(self, *args):
        """
        The add method...:

        Adds a new Atom of kind elt (string) on the carbon nc in the branch nb.
        Atoms added this way are not considered as being part of the branch they are bounded to and aren't considered a new branch of the molecule.
        """
        if self.locked:
            raise LockedMolecule
        for nc, nb, elt in args:
            atom = Atom(elt, self.atoms[-1].id + 1)
            self.__branches[nb - 1][nc - 1].bond(atom)
            self.atoms.append(atom)

        return self

    def mutate(self, *args):
        """
        m.mutate((nc,nb,elt), ...)
        The mutate method...:

        Mutates the carbon nc in the branch nb to the chemical element elt(given as a string).
        Don't forget that carbons and branches are 1-indexed.
        This is mutation: the id number of the Atom instance stays the same. See the Atom class specs about that.
        """
        if self.locked:
            raise LockedMolecule
        for nc, nb, elt in args:
            self.__branches[nb - 1][nc - 1].mutate(elt)

        return self

    def closer(self):
        """
        The closer method...:

        Finalizes the molecule instance, adding missing hydrogens everywhere and locking the object.
        """

        for nb in range(len(self.__branches)):
            for na in range(len(self.__branches[nb])):
                self.add(
                    *([(na + 1, nb + 1, "H")] * self.__branches[nb][na].valence_free)
                )

        self.locked = True
        return self

    def bounder(self, *args):
        """
        m.bounder((c1,b1,c2,b2), ...)
        The bounder method...:

        Creates new bounds between two atoms of existing branches.
        Each argument is a tuple (python), array (ruby/JS), or T object (java) of four integers giving:
        c1 & b1: carbon and branch positions of the first atom
        c2 & b2: carbon and branch positions of the second atom
        All positions are 1-indexed, meaning (1,1,5,3) will bound the first carbon of the first branch with the fifth of the third branch.
        Only positive integers will be used.
        """
        if self.locked:
            raise LockedMolecule
        for c1, b1, c2, b2 in args:
            try:
                self.__branches[b1 - 1][c1 - 1].bond(self.__branches[b2 - 1][c2 - 1])
            except:
                raise InvalidBond
        return self

    @property
    def formula(self) -> str:
#         if not self.locked:
#             raise UnlockedMolecule
        atoms = dict()
        for atom in self.atoms:
            atoms[atom.element] = atoms.get(atom.element, 0) + 1

        formula = (
            (
                f'C{atoms["C"] if atoms["C"] > 1 else ""}'
                if atoms.get("C", 0) > 0
                else ""
            )
            + (
                f'H{atoms["H"] if atoms["H"] > 1 else ""}'
                if atoms.get("H", 0) > 0
                else ""
            )
            + (
                f'O{atoms["O"] if atoms["O"] > 1 else ""}'
                if atoms.get("O", 0) > 0
                else ""
            )
        )
        atoms.pop("C", None)
        atoms.pop("H", None)
        atoms.pop("O", None)
        atoms = sorted(atoms.items(), key=lambda x: x[0])
        for elt, c in atoms:
            formula += f"{elt if  c > 0 else ''}{c if c > 1  else ''}"
        self.__formula = formula
        return formula

    @property
    def molecular_weight(self):
#         if not self.locked:
#             raise UnlockedMolecule
        return sum(atom.weight for atom in self.atoms)

    def add_chaining(self, nc, nb, *elements):
        """m.add_chaining(nc, nb, elt1, elt2, ...)
        The add_chaining method...:

        Adds on the carbon nc in the branch nb a chain with all the provided elements,
        in the specified order. Meaning: m.add_chaining(2, 5, "N", "C", "C", "Mg", "Br")
        will add the chain ...-N-C-C-Mg-Br to the atom number 2 in the branch 5.
        As for the add method, this chain is not considered a new branch of the molecule.
        """
        if self.locked:
            raise LockedMolecule
        if any(elt in ["H",'F',"Cl","Br"] for elt in elements[1:-1]):
            raise InvalidBond
        try:
            mol = deepcopy(self)
            # chain = [(nc, nb, elt) for elt in elements]
            # mol.__branches[nb-1][nc-1]
            atom = Atom(elements[0], mol.atoms[-1].id + 1)
            mol.__branches[nb - 1][nc - 1].bond(atom)
            mol.atoms.append(atom)
            mol.__branches[nb - 1].append(atom)
            for elt in elements[1:]:
                atom = Atom(elt, mol.atoms[-1].id + 1)
                mol.__branches[nb - 1][-1].bond(atom)
                mol.atoms.append(atom)
                mol.__branches[nb - 1].append(atom)

            return mol

        except:
            return self

    def unlock(self):
        """The unlock method...:

        Makes the molecule mutable again.

        Hydrogens should be removed, as well as any empty branch you might encounter during the process.

        After the molecule has been "unlocked", if by any (bad...) luck it does not have any branch left, 
        throw an EmptyMolecule exception.

        The id numbers of the remaining atoms have to be continuous again (beginning at 1), 
        keeping the order they had when the molecule was locked.

        After removing hydrogens, if you end up with some atoms 
        that aren't connected in any way to the branches of the unlocked molecule, keep them anyway in the Molecule instance (for the sake of simplicity...). Note that all the branches following removed one are shifted one step back (ex: removing the third branch, the "previously fourth branch" will be considered the third branch of the molecule in subsequent operations).
        
        Once unlocked, the molecule has to be modifiable again, in any manner.
         """
        self.locked = False

        for branch in self.__branches:
            for atom in branch:
                if atom.element == "H":
                    atom.unbond()
                    branch.remove(atom)
                    self.atoms.remove(atom)
        

        return self

In [54]:


conf = (
    """ 1,1-dimethyl-2-propylcyclohexane:
            CH3   CH3
               \ /
CH3-CH2-CH2-CH2-CH2-CH2
            |       |
            CH2-CH2-CH2
""",
    "2-propyl-1,1-dimethylcyclohexane",
    [9, 1, 1],
    [(4, 1, 9, 1), (5, 1, 1, 2), (5, 1, 1, 3)],
    "C11H22",
    154,
    [
        "Atom(C.1: C2,H,H,H)",
        "Atom(C.2: C1,C3,H,H)",
        "Atom(C.3: C2,C4,H,H)",
        "Atom(C.4: C3,C5,C9,H)",
        "Atom(C.5: C4,C6,C10,C11)",
        "Atom(C.6: C5,C7,H,H)",
        "Atom(C.7: C6,C8,H,H)",
        "Atom(C.8: C7,C9,H,H)",
        "Atom(C.9: C4,C8,H,H)",
        "Atom(C.10: C5,H,H,H)",
        "Atom(C.11: C5,H,H,H)",
    ],
)


m = Molecule(conf[1]).brancher(*conf[2]).bounder(*conf[3]).closer()
print(m.formula, m.molecular_weight)
# print(m.atoms[0])
for i in range(len(m.atoms)):
    if i == len(conf[6]):
        break
    print(f"{str(m.atoms[i]):25} == {conf[6][i]}\t= {str(m.atoms[i]) == conf[6][i]}")
m.add()


C11H22 154.0
Atom(C.1: C2,H,H,H)       == Atom(C.1: C2,H,H,H)	= True
Atom(C.2: C1,C3,H,H)      == Atom(C.2: C1,C3,H,H)	= True
Atom(C.3: C2,C4,H,H)      == Atom(C.3: C2,C4,H,H)	= True
Atom(C.4: C3,C5,C9,H)     == Atom(C.4: C3,C5,C9,H)	= True
Atom(C.5: C4,C6,C10,C11)  == Atom(C.5: C4,C6,C10,C11)	= True
Atom(C.6: C5,C7,H,H)      == Atom(C.6: C5,C7,H,H)	= True
Atom(C.7: C6,C8,H,H)      == Atom(C.7: C6,C8,H,H)	= True
Atom(C.8: C7,C9,H,H)      == Atom(C.8: C7,C9,H,H)	= True
Atom(C.9: C4,C8,H,H)      == Atom(C.9: C4,C8,H,H)	= True
Atom(C.10: C5,H,H,H)      == Atom(C.10: C5,H,H,H)	= True
Atom(C.11: C5,H,H,H)      == Atom(C.11: C5,H,H,H)	= True


In [49]:


conf = (
    """ 1,1-dimethyl-2-propylcyclohexane:
            CH3   CH3
               \ /
CH3-CH2-CH2-CH2-CH2-CH2
            |       |
            CH2-CH2-CH2
""",
    "2-propyl-1,1-dimethylcyclohexane",
    [9, 1, 1],
    [(4, 1, 9, 1), (5, 1, 1, 2), (5, 1, 1, 3)],
    "C11H22",
    154,
    [
        "Atom(C.1: C2,H,H,H)",
        "Atom(C.2: C1,C3,H,H)",
        "Atom(C.3: C2,C4,H,H)",
        "Atom(C.4: C3,C5,C9,H)",
        "Atom(C.5: C4,C6,C10,C11)",
        "Atom(C.6: C5,C7,H,H)",
        "Atom(C.7: C6,C8,H,H)",
        "Atom(C.8: C7,C9,H,H)",
        "Atom(C.9: C4,C8,H,H)",
        "Atom(C.10: C5,H,H,H)",
        "Atom(C.11: C5,H,H,H)",
    ],
)


m = Molecule(conf[1]).brancher(*conf[2]).bounder(*conf[3])
m.bounder((1, 1, 1, 1))


InvalidBond: 

In [15]:
chr(ord("a") )

'@'

In [46]:
raise InvalidBond

InvalidBond: 